Import stuff

In [1]:
from pymongo import MongoClient
from pprint import pprint
from datetime import datetime, timedelta
from time import sleep
from json import dumps
from kafka3 import KafkaProducer
import random
import csv

In [2]:
ip_address = "YOUR IP"
client = MongoClient(f"{ip_address}", 27017)
db = client.fit3182_assignment_db
collection = db.climate_hotspot
query = collection.aggregate([{"$project" : {"_id" : 0, "date" : 1}}, {"$sort" : {"date" : -1}}, {"$limit" : 1}])
start_date = query.__next__()["date"]

In [3]:
start_date

datetime.datetime(2023, 1, 1, 0, 0)

Now we can load the data from hotspot_AQUA_streaming.csv

In [4]:
aqua_data = []
with open("hotspot_AQUA_streaming.csv", "r") as aqua_file:
    for aqua in csv.DictReader(aqua_file):
        ##change some data types
        aqua["latitude"] = float(aqua["latitude"])
        aqua["longitude"] = float(aqua["longitude"])
        aqua["confidence"] = int(aqua["confidence"])
        aqua["surface_temperature_celcius"] = int(aqua["surface_temperature_celcius"])
        aqua_data.append(aqua)

Now to see a preview of the data to pass to the producer

In [5]:
data = {"datetime" : str(start_date.strftime("%d/%m/%Y %H:%M:%S")), "aqua" : aqua_data[random.randrange(0,len(aqua_data))]}

In [6]:
data

{'datetime': '01/01/2023 00:00:00',
 'aqua': {'latitude': -36.1345,
  'longitude': 146.351,
  'confidence': 98,
  'surface_temperature_celcius': 84}}

Now setup the kafka producer

In [7]:
host_ip = "YOUR IP"

def publish_message(producer_instance, topic_name, key, data):
    try:
        key_bytes = bytes(key, encoding='utf-8')
        producer_instance.send(topic_name, key=key_bytes, value=data)
        producer_instance.flush()
        print('Message published successfully. Data: ' + str(data))
    except Exception as ex:
        print('Exception in publishing message.')
        print(str(ex))
        
def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=[f'{host_ip}:9092'],
                                  value_serializer=lambda x:dumps(x).encode('ascii'),
                                  api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka.')
        print(str(ex))
    finally:
        return _producer

In [8]:
latest_date = datetime(start_date.year, start_date.month, start_date.day)
topic = 'Aqua'

print('Publishing records..')
producer = connect_kafka_producer()

while True:
    data = {"datetime" : str(latest_date.strftime("%d/%m/%Y %H:%M:%S")), "aqua" : aqua_data[random.randrange(0,len(aqua_data))]}
    publish_message(producer, topic, 'jsondata', data)
    latest_date = latest_date + timedelta(hours=4, minutes = 48) ##2 seconds is 4 hours 48 minutes
    sleep(2)

Publishing records..
Message published successfully. Data: {'datetime': '01/01/2023 00:00:00', 'aqua': {'latitude': -36.3579, 'longitude': 146.1268, 'confidence': 63, 'surface_temperature_celcius': 42}}
Message published successfully. Data: {'datetime': '01/01/2023 04:48:00', 'aqua': {'latitude': -36.5586, 'longitude': 144.0423, 'confidence': 92, 'surface_temperature_celcius': 71}}
Message published successfully. Data: {'datetime': '01/01/2023 09:36:00', 'aqua': {'latitude': -36.3652, 'longitude': 141.3508, 'confidence': 66, 'surface_temperature_celcius': 43}}
Message published successfully. Data: {'datetime': '01/01/2023 14:24:00', 'aqua': {'latitude': -37.3511, 'longitude': 149.4088, 'confidence': 61, 'surface_temperature_celcius': 46}}
Message published successfully. Data: {'datetime': '01/01/2023 19:12:00', 'aqua': {'latitude': -36.7806, 'longitude': 141.5597, 'confidence': 94, 'surface_temperature_celcius': 74}}
Message published successfully. Data: {'datetime': '02/01/2023 00:00:

Message published successfully. Data: {'datetime': '10/01/2023 00:00:00', 'aqua': {'latitude': -36.6963, 'longitude': 141.8645, 'confidence': 79, 'surface_temperature_celcius': 52}}
Message published successfully. Data: {'datetime': '10/01/2023 04:48:00', 'aqua': {'latitude': -34.8544, 'longitude': 143.1831, 'confidence': 74, 'surface_temperature_celcius': 48}}
Message published successfully. Data: {'datetime': '10/01/2023 09:36:00', 'aqua': {'latitude': -37.8475, 'longitude': 147.2512, 'confidence': 76, 'surface_temperature_celcius': 50}}
Message published successfully. Data: {'datetime': '10/01/2023 14:24:00', 'aqua': {'latitude': -36.6867, 'longitude': 141.137, 'confidence': 84, 'surface_temperature_celcius': 59}}
Message published successfully. Data: {'datetime': '10/01/2023 19:12:00', 'aqua': {'latitude': -36.3588, 'longitude': 141.5272, 'confidence': 65, 'surface_temperature_celcius': 42}}
Message published successfully. Data: {'datetime': '11/01/2023 00:00:00', 'aqua': {'latitud

Message published successfully. Data: {'datetime': '19/01/2023 04:48:00', 'aqua': {'latitude': -37.6646, 'longitude': 142.9131, 'confidence': 82, 'surface_temperature_celcius': 56}}
Message published successfully. Data: {'datetime': '19/01/2023 09:36:00', 'aqua': {'latitude': -36.714, 'longitude': 141.1485, 'confidence': 61, 'surface_temperature_celcius': 41}}
Message published successfully. Data: {'datetime': '19/01/2023 14:24:00', 'aqua': {'latitude': -34.2432, 'longitude': 142.0625, 'confidence': 79, 'surface_temperature_celcius': 52}}
Message published successfully. Data: {'datetime': '19/01/2023 19:12:00', 'aqua': {'latitude': -36.1057, 'longitude': 141.7608, 'confidence': 100, 'surface_temperature_celcius': 120}}
Message published successfully. Data: {'datetime': '20/01/2023 00:00:00', 'aqua': {'latitude': -36.8346, 'longitude': 141.3552, 'confidence': 86, 'surface_temperature_celcius': 61}}
Message published successfully. Data: {'datetime': '20/01/2023 04:48:00', 'aqua': {'latit

Message published successfully. Data: {'datetime': '28/01/2023 09:36:00', 'aqua': {'latitude': -36.2913, 'longitude': 142.9408, 'confidence': 80, 'surface_temperature_celcius': 54}}
Message published successfully. Data: {'datetime': '28/01/2023 14:24:00', 'aqua': {'latitude': -37.0761, 'longitude': 141.0574, 'confidence': 100, 'surface_temperature_celcius': 89}}
Message published successfully. Data: {'datetime': '28/01/2023 19:12:00', 'aqua': {'latitude': -38.2234, 'longitude': 143.9549, 'confidence': 56, 'surface_temperature_celcius': 39}}
Message published successfully. Data: {'datetime': '29/01/2023 00:00:00', 'aqua': {'latitude': -36.9521, 'longitude': 143.0574, 'confidence': 73, 'surface_temperature_celcius': 47}}
Message published successfully. Data: {'datetime': '29/01/2023 04:48:00', 'aqua': {'latitude': -38.0942, 'longitude': 146.9603, 'confidence': 52, 'surface_temperature_celcius': 39}}
Message published successfully. Data: {'datetime': '29/01/2023 09:36:00', 'aqua': {'latit

Message published successfully. Data: {'datetime': '06/02/2023 14:24:00', 'aqua': {'latitude': -37.8067, 'longitude': 141.7598, 'confidence': 57, 'surface_temperature_celcius': 40}}
Message published successfully. Data: {'datetime': '06/02/2023 19:12:00', 'aqua': {'latitude': -37.3656, 'longitude': 143.8206, 'confidence': 59, 'surface_temperature_celcius': 40}}
Message published successfully. Data: {'datetime': '07/02/2023 00:00:00', 'aqua': {'latitude': -37.164, 'longitude': 148.915, 'confidence': 55, 'surface_temperature_celcius': 42}}
Message published successfully. Data: {'datetime': '07/02/2023 04:48:00', 'aqua': {'latitude': -36.4469, 'longitude': 144.7952, 'confidence': 78, 'surface_temperature_celcius': 51}}
Message published successfully. Data: {'datetime': '07/02/2023 09:36:00', 'aqua': {'latitude': -37.561, 'longitude': 148.032, 'confidence': 73, 'surface_temperature_celcius': 34}}
Message published successfully. Data: {'datetime': '07/02/2023 14:24:00', 'aqua': {'latitude':

Message published successfully. Data: {'datetime': '15/02/2023 19:12:00', 'aqua': {'latitude': -37.6387, 'longitude': 142.9032, 'confidence': 87, 'surface_temperature_celcius': 88}}
Message published successfully. Data: {'datetime': '16/02/2023 00:00:00', 'aqua': {'latitude': -37.1378, 'longitude': 141.9545, 'confidence': 81, 'surface_temperature_celcius': 54}}
Message published successfully. Data: {'datetime': '16/02/2023 04:48:00', 'aqua': {'latitude': -36.4315, 'longitude': 141.1987, 'confidence': 76, 'surface_temperature_celcius': 49}}
Message published successfully. Data: {'datetime': '16/02/2023 09:36:00', 'aqua': {'latitude': -36.8705, 'longitude': 142.2896, 'confidence': 76, 'surface_temperature_celcius': 50}}
Message published successfully. Data: {'datetime': '16/02/2023 14:24:00', 'aqua': {'latitude': -36.1122, 'longitude': 142.1491, 'confidence': 90, 'surface_temperature_celcius': 76}}
Message published successfully. Data: {'datetime': '16/02/2023 19:12:00', 'aqua': {'latitu

Message published successfully. Data: {'datetime': '25/02/2023 00:00:00', 'aqua': {'latitude': -36.1345, 'longitude': 146.351, 'confidence': 98, 'surface_temperature_celcius': 84}}
Message published successfully. Data: {'datetime': '25/02/2023 04:48:00', 'aqua': {'latitude': -37.47, 'longitude': 148.089, 'confidence': 83, 'surface_temperature_celcius': 50}}
Message published successfully. Data: {'datetime': '25/02/2023 09:36:00', 'aqua': {'latitude': -38.4685, 'longitude': 146.673, 'confidence': 59, 'surface_temperature_celcius': 40}}
Message published successfully. Data: {'datetime': '25/02/2023 14:24:00', 'aqua': {'latitude': -37.379, 'longitude': 148.132, 'confidence': 62, 'surface_temperature_celcius': 78}}
Message published successfully. Data: {'datetime': '25/02/2023 19:12:00', 'aqua': {'latitude': -36.3606, 'longitude': 144.498, 'confidence': 84, 'surface_temperature_celcius': 58}}
Message published successfully. Data: {'datetime': '26/02/2023 00:00:00', 'aqua': {'latitude': -35

Message published successfully. Data: {'datetime': '06/03/2023 04:48:00', 'aqua': {'latitude': -36.0396, 'longitude': 141.7183, 'confidence': 95, 'surface_temperature_celcius': 76}}
Message published successfully. Data: {'datetime': '06/03/2023 09:36:00', 'aqua': {'latitude': -36.5234, 'longitude': 144.6939, 'confidence': 62, 'surface_temperature_celcius': 41}}
Message published successfully. Data: {'datetime': '06/03/2023 14:24:00', 'aqua': {'latitude': -37.0038, 'longitude': 142.1629, 'confidence': 68, 'surface_temperature_celcius': 44}}
Message published successfully. Data: {'datetime': '06/03/2023 19:12:00', 'aqua': {'latitude': -36.9194, 'longitude': 143.6131, 'confidence': 76, 'surface_temperature_celcius': 50}}
Message published successfully. Data: {'datetime': '07/03/2023 00:00:00', 'aqua': {'latitude': -37.579, 'longitude': 144.1483, 'confidence': 55, 'surface_temperature_celcius': 39}}
Message published successfully. Data: {'datetime': '07/03/2023 04:48:00', 'aqua': {'latitud

Message published successfully. Data: {'datetime': '15/03/2023 09:36:00', 'aqua': {'latitude': -37.7714, 'longitude': 142.8339, 'confidence': 77, 'surface_temperature_celcius': 50}}
Message published successfully. Data: {'datetime': '15/03/2023 14:24:00', 'aqua': {'latitude': -36.1532, 'longitude': 143.5527, 'confidence': 100, 'surface_temperature_celcius': 90}}
Message published successfully. Data: {'datetime': '15/03/2023 19:12:00', 'aqua': {'latitude': -36.4309, 'longitude': 141.1436, 'confidence': 80, 'surface_temperature_celcius': 54}}
Message published successfully. Data: {'datetime': '16/03/2023 00:00:00', 'aqua': {'latitude': -36.9318, 'longitude': 143.0907, 'confidence': 100, 'surface_temperature_celcius': 122}}
Message published successfully. Data: {'datetime': '16/03/2023 04:48:00', 'aqua': {'latitude': -38.3998, 'longitude': 147.064, 'confidence': 89, 'surface_temperature_celcius': 65}}
Message published successfully. Data: {'datetime': '16/03/2023 09:36:00', 'aqua': {'lati

KeyboardInterrupt: 